In [2]:
from tensorflow.keras import layers, models, backend, utils, metrics
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
# from kaggle_datasets import KaggleDatasets
# GCS_DS_PATH = KaggleDatasets().get_gcs_path()
# GCS_DS_PATH


In [4]:
from tensorflow.keras import preprocessing
base_path = '/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/'

train_flow = preprocessing.image_dataset_from_directory(
    base_path + 'train/',
    image_size=(224, 224),
    batch_size=64,
    labels='inferred',
    label_mode='binary'
)
test_flow = preprocessing.image_dataset_from_directory(
    base_path + 'test/',
    image_size=(224, 224),
    batch_size=64,
    labels='inferred',
    label_mode='binary'
)
valid_flow = preprocessing.image_dataset_from_directory(
    base_path + 'valid/',
    image_size=(224, 224),
    batch_size=64,
    labels='inferred',
    label_mode='binary'
)
train_flow.class_names

In [5]:
# from tensorflow import distribute
# # detect and init the TPU
# tpu = distribute.cluster_resolver.TPUClusterResolver.connect()

# # instantiate a distribution strategy
# tpu_strategy = distribute.experimental.TPUStrategy(tpu)

In [6]:

# with tpu_strategy.scope():
model = models.Sequential()

model.add(
    layers.Conv2D(32, (3, 3),activation='relu', input_shape=(224, 224, 3))
)

model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

model.compile(
    optimizer = 'adam', 
    loss = 'binary_crossentropy', 
    metrics = [
        'accuracy'
    ]
)

In [7]:
train_steps = 100000//64
valid_steps = 20000//64

hist = model.fit(
    train_flow,
    epochs=20,
    validation_data=valid_flow,
    validation_steps=valid_steps
)

In [16]:
models.save_model(model,"/kaggle/working/model")

In [ ]:
y_pred = model.predict(test_flow)
y_pred


In [10]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics

In [11]:
y_test = test_flow.class_names

In [12]:
fig, ax = plt.subplots(1, 4, figsize=(20, 3))
ax = ax.ravel()

for i, met in enumerate([ "accuracy", "loss"]):
    ax[i].plot(hist.history[met])
    ax[i].plot(hist.history["val_" + met])
    ax[i].set_title("Model {}".format(met))
    ax[i].set_xlabel("epochs")
    ax[i].set_ylabel(met)
    ax[i].legend(["train", "val"])

In [13]:
plt.plot(hist.history["accuracy"])
plt.plot(hist.history["val_accuracy"])
plt.title("Model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Train", "Test"])
plt.show()

In [14]:
# test_flow
# model.evaluate(test_flow)